In [ ]:
'''
Product ID
Store ID
The type of store where the supplement was sold
The type of location the order was received from
Sales Date
Region code
Whether it is a public holiday or not at the time of order
Whether the product was on discount or not
Number of orders placed
Sales
Datalink: https://thecleverprogrammer.com/2021/09/27/number-of-orders-prediction-with-machine-learning/
datasource: https://raw.githubusercontent.com/amankharwal/Website-data/master/supplement.csv
'''


In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("supplement.csv")
df = df.sample(50000)


In [2]:
# EDA

In [3]:
df.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
112577,T1112578,128,S1,L2,R2,2018-11-05,0,No,45,28389.00
91100,T1091101,25,S1,L3,R1,2018-09-07,0,Yes,68,41949.00
82552,T1082553,319,S3,L1,R2,2018-08-15,1,No,33,19031.79
118587,T1118588,64,S3,L1,R2,2018-11-21,1,No,62,37486.62
55760,T1055761,99,S1,L1,R3,2018-06-02,0,Yes,76,52503.00


In [4]:
df.tail()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
127555,T1127556,167,S1,L1,R2,2018-12-16,0,No,60,41727.0
16992,T1016993,35,S1,L1,R2,2018-02-16,0,No,63,37998.0
125634,T1125635,111,S1,L1,R2,2018-12-11,0,No,62,38682.0
24031,T1024032,331,S1,L1,R4,2018-03-07,0,No,63,40893.0
37464,T1037465,312,S4,L2,R1,2018-04-13,0,No,153,80808.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 112577 to 37464
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             50000 non-null  object 
 1   Store_id       50000 non-null  int64  
 2   Store_Type     50000 non-null  object 
 3   Location_Type  50000 non-null  object 
 4   Region_Code    50000 non-null  object 
 5   Date           50000 non-null  object 
 6   Holiday        50000 non-null  int64  
 7   Discount       50000 non-null  object 
 8   #Order         50000 non-null  int64  
 9   Sales          50000 non-null  float64
dtypes: float64(1), int64(3), object(6)
memory usage: 4.2+ MB


In [6]:
df.drop(['ID','Date'],axis = 1, inplace= True)

In [7]:
df['Holiday'].unique()

array([0, 1], dtype=int64)

In [8]:
d = {'Yes':1, 'No':0}
df['Discount'] = df['Discount'].map(d)

In [9]:
y = df['#Order']
x = df.drop(['#Order'], axis=1)

In [10]:
x= pd.get_dummies(x,drop_first=True)

In [11]:
def fnc_all_regression_models(x,y):
    
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import Ridge,Lasso
    from sklearn.linear_model import ElasticNet
    from sklearn.tree import ExtraTreeRegressor
    from sklearn.ensemble import GradientBoostingRegressor
    from sklearn.neighbors import KNeighborsRegressor
    
    from sklearn.model_selection import train_test_split

    from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.pipeline import Pipeline
    from sklearn import svm
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.ensemble import RandomForestRegressor
    
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

    L=LinearRegression()
    R=Ridge()
    Lass=Lasso()
    E=ElasticNet()
    ExTree=ExtraTreeRegressor()
    GBR=GradientBoostingRegressor()
    KN=KNeighborsRegressor()
    PLR = Pipeline([('poly', PolynomialFeatures(degree=2)),
        ('linear', LinearRegression(fit_intercept=True))])
    SVM = svm.SVR()
    DTR = DecisionTreeRegressor(max_depth=12)
    RFR=RandomForestRegressor(n_estimators=100, max_features=100, max_leaf_nodes=100,random_state=42)

    algos=[L,R,Lass,E,ExTree,GBR,KN,PLR, SVM, DTR,RFR]
    algo_names=['LinearRegression','Ridge','Lasso','ElasticNet','ExtraTreeRegressor',
                'GradientBoostingRegressor','KNeighborsRegressor','Polynomial',
                'Support Vector Regression', 'Decision Tree Regressor','Random Forest Regressor']
    r_squared=[]
    rmse=[]
    mae=[]
    
    result = pd.DataFrame(columns=['R_Squared','RMSE','MAE'],index=algo_names)
    
    for item in algos:
        item.fit(x_train,y_train)
        item.predict(x_test)
        r_squared.append(r2_score(y_test,item.predict(x_test)))
        rmse.append((mean_squared_error(y_test,item.predict(x_test)))**.5)
        mae.append(mean_absolute_error(y_test,item.predict(x_test)))
        
    result.R_Squared=r_squared
    result.RMSE=rmse
    result.MAE=mae
        
    return result.sort_values('R_Squared',ascending=False)

In [12]:
fnc_all_regression_models(x,y)

,R_Squared,RMSE,MAE
GradientBoostingRegressor,0.942737,7.293437,5.250887
Polynomial,0.942089,7.334601,5.330273
Decision Tree Regressor,0.937872,7.596952,5.330788
Random Forest Regressor,0.935558,7.737147,5.656104
Ridge,0.930809,8.017192,5.815764
LinearRegression,0.930808,8.017209,5.815904
ExtraTreeRegressor,0.913553,8.961299,6.050950
Lasso,0.905563,9.366304,6.894247
ElasticNet,0.897387,9.763329,7.158147
Support Vector Regression,0.881028,10.512810,7.563750
